In [1]:
from lxml import html  
import csv,os,json
import requests
# from exceptions import ValueError
from time import sleep

def AmzonParser(url):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64)    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/42.0.2311.90 Safari/537.36'}
    page = requests.get(url,headers=headers)
    while True:
        sleep(3)
        try:
            doc = html.fromstring(page.content)
            XPATH_NAME = '//h1[@id="title"]//text()'
            XPATH_SALE_PRICE = '//span[contains(@id,"ourprice") or contains(@id,"saleprice")]/text()'
            XPATH_ORIGINAL_PRICE = '//td[contains(text(),"List Price") or contains(text(),"M.R.P") or contains(text(),"Price")]/following-sibling::td/text()'
            XPATH_CATEGORY = '//a[@class="a-link-normal a-color-tertiary"]//text()'
            XPATH_AVAILABILITY = '//div[@id="availability"]//text()'

            RAW_NAME = doc.xpath(XPATH_NAME)
            RAW_SALE_PRICE = doc.xpath(XPATH_SALE_PRICE)
            RAW_CATEGORY = doc.xpath(XPATH_CATEGORY)
            RAW_ORIGINAL_PRICE = doc.xpath(XPATH_ORIGINAL_PRICE)
            RAw_AVAILABILITY = doc.xpath(XPATH_AVAILABILITY)

            NAME = ' '.join(''.join(RAW_NAME).split()) if RAW_NAME else None
            SALE_PRICE = ' '.join(''.join(RAW_SALE_PRICE).split()).strip() if RAW_SALE_PRICE else None
            CATEGORY = ' > '.join([i.strip() for i in RAW_CATEGORY]) if RAW_CATEGORY else None
            ORIGINAL_PRICE = ''.join(RAW_ORIGINAL_PRICE).strip() if RAW_ORIGINAL_PRICE else None
            AVAILABILITY = ''.join(RAw_AVAILABILITY).strip() if RAw_AVAILABILITY else None

            if not ORIGINAL_PRICE:
                ORIGINAL_PRICE = SALE_PRICE

            if page.status_code!=200:
                raise ValueError('captha')
            data = {
                    'NAME':NAME,
                    'SALE_PRICE':SALE_PRICE,
                    'CATEGORY':CATEGORY,
                    'ORIGINAL_PRICE':ORIGINAL_PRICE,
                    'AVAILABILITY':AVAILABILITY,
                    'URL':url,
                    }

            return data
        except Exception as e:
            print(e)

def ReadAsin():
# AsinList = csv.DictReader(open(os.path.join(os.path.dirname(__file__),"Asinfeed.csv")))
    AsinList = ['B0046UR4F4',
#                 'B00JGTVU5A',
#                 'B00GJYCIVK',
#                 'B00EPGK7CQ',
#                 'B00EPGKA4G',
#                 'B00YW5DLB4',
#                 'B00KGD0628',
#                 'B00O9A48N2',
#                 'B00O9A4MEW',
                'B00UZKG8QU',]
    extracted_data = []
    for i in AsinList:
        url = "http://www.amazon.com/dp/"+i
        print("Processing: "+ url)
        extracted_data.append(AmzonParser(url))
        sleep(5)
    f=open('data.json','w')
    json.dump(extracted_data,f,indent=4)


if __name__ == "__main__":
    ReadAsin()

Processing: http://www.amazon.com/dp/B0046UR4F4
Processing: http://www.amazon.com/dp/B00UZKG8QU


In [2]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('max_colwidth', 600)
pd.read_json('data.json')

,NAME,SALE_PRICE,CATEGORY,ORIGINAL_PRICE,AVAILABILITY,URL
0,G-Technology G-SPEED eS PRO High-Performance Fail-Safe RAID Solution for HD/2K Production 8TB (0G01873),None,Electronics > Computers & Accessories > Data Storage > External Hard Drives,None,Available from these sellers.,http://www.amazon.com/dp/B0046UR4F4
1,G-Technology G-RAID USB Removable Dual Drive Storage System 8TB (0G04069),$529.95,Electronics > Computers & Accessories > Data Storage,$529.95,In stock.,http://www.amazon.com/dp/B00UZKG8QU


In [21]:
countries=['com','co.uk','ca','de']
books=[
        '''http://www.amazon.%s/Glass-House-Climate-Millennium-ebook/dp/B005U3U69C''',
        '''http://www.amazon.%s/The-Japanese-Observer-ebook/dp/B0078FMYD6''',
        '''http://www.amazon.%s/Falling-Through-Water-ebook/dp/B009VJ1622''',
      ]
import urllib3
for book in books:
#     print('-'*40)
#     print(book.split('%s/')[1])
    for country in countries:
        asin=book.split('/')[-1]; title=book.split('/')[3]
        url='''http://www.amazon.%s/product-reviews/%s'''%(country,asin)
        try: f = urllib3.PoolManager(url)
        except: 
            page=""
            page=f.read().lower()
            print('%s=%s'%(country, page.count('member-review')))
# print('-'*40)

----------------------------------------
Glass-House-Climate-Millennium-ebook/dp/B005U3U69C
----------------------------------------
The-Japanese-Observer-ebook/dp/B0078FMYD6
----------------------------------------
Falling-Through-Water-ebook/dp/B009VJ1622
----------------------------------------


In [9]:
import sys
! pip install urllib3